In [ ]:
%pylab inline
from astropy.io import fits
import pandas as pd

In [ ]:
non_AGN_list = np.loadtxt('../../Astro5_Mount/Anthony/Text_Files/nonAGN.txt')

seyfert_before = np.loadtxt('../../Astro5_Mount/Anthony/Text_Files/seyfert_before.txt')
sf_before = np.loadtxt('../../Astro5_Mount/Anthony/Text_Files/sf_before.txt')
liner_before = np.loadtxt('../../Astro5_Mount/Anthony/Text_Files/liner_before.txt')

AGN_list = np.append(seyfert_before, np.append(sf_before,liner_before,axis=0),axis=0)
full_list = np.append(non_AGN_list, AGN_list,axis=0)

In [ ]:
def retrieve_map(galaxies, filename):
    map_vals = []
    for gal in galaxies:
        plateid = str(int(gal[0]))
        ifuid = str(int(gal[1]))
        
        path = "../../Astro5_Mount/Anthony/Fits_Files/MPL_5/"+plateid+"/"+ifuid+"/"+plateid+"-"+ifuid+ "-" + filename
        currentHdulist = fits.open(path)
        vals = currentHdulist[0].data
        
        SNRPath = "../../Astro5_Mount/Anthony/Fits_Files/MPL_5/"+plateid+"/"+ifuid+"/"+plateid+"-"+ifuid+ "-SNR_Map_Final.fits"
        currentHdulist = fits.open(SNRPath)
        SNRVals = currentHdulist[0].data
        
        nonNan = vals[~np.isnan(vals)]
        SNRVals = SNRVals[~np.isnan(vals)]
        vals = nonNan[SNRVals > 10]
        if np.size(vals) != 0:
            map_vals.append(np.percentile(vals,75))
        else:
            map_vals.append(np.nan)
    return map_vals

# Add luminosity to Dataframe

In [ ]:
def retrieve_luminosity(plateid, ifuid, percentile):
    drpall = fits.open('../../Astro5_Mount/drpall-v2_0_1.fits')
    data = drpall[1].data
    plates = np.where(data.plate == int(plateid))
    for plate in plates[0]:
        if data[plate]['ifudsgn'] == str(ifuid):
            luminosities = data[plate]['nsa_z']
            return np.percentile(luminosities, percentile)
    return np.nan

In [ ]:
lums = []
for gal in full_list:
    plateid = str(int(gal[0]))
    ifuid = str(int(gal[1]))
    gal_lum_75 = retrieve_luminosity(plateid, ifuid, percentile)
    lums.append(gal_lum_75)

### Flux (Using 75th percentile)